In [1]:
# 复现paragraph vector model
# 2022.09.12

# Paragraph Vector

## PV-DM

In [ ]:
# word2vec-CBOW在输入层增加了一个paragraph token/id
# website: https://blog.csdn.net/qq_28790663/article/details/115597384

In [4]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from gensim.utils import simple_preprocess
from smart_open import open

In [8]:
file_name = './data/pv_dm_data/proasmdataset.txt'
train_vec = 'proasmdatasetVec.txt.model'

In [9]:
def read_corpus(filename, tokens_only=False):
    with open(filename, encoding='utf-8') as f:
        for i,line in enumerate(f):
            tokens = simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                yield TaggedDocument(tokens, [i])

In [10]:
train_corpus = list(read_corpus(file_name))
test_corpus = list(read_corpus(file_name, tokens_only=True))

In [12]:
train_corpus

[TaggedDocument(words=['mov', 'eax', 'esp', 'arg_', 'sub', 'esp', 'ch', 'push', 'esi', 'push', 'edi', 'mov', 'edi', 'ecx', 'push', 'ch', 'mov', 'edi', 'eax', 'lea', 'eax', 'esp', 'var_', 'push', 'push', 'eax', 'mov', 'dword', 'ptr', 'edi', 'call', 'mov', 'esi', 'esp', 'arg_', 'add', 'esp', 'ch', 'cmp', 'esi', 'fh'], tags=[0]),
 TaggedDocument(words=['push', 'fh', 'push', 'esi', 'push', 'offset', 'atoomanylinkpro', 'push', 'offset', 'adevcolorengine', 'push', 'push', 'push', 'call', 'sub_', 'dfb', 'add', 'esp', 'ch'], tags=[1]),
 TaggedDocument(words=['xor', 'ecx', 'ecx', 'test', 'esi', 'esi'], tags=[2]),
 TaggedDocument(words=['mov', 'edx', 'esp', 'arg_', 'lea', 'eax', 'esp', 'var_', 'sub', 'edx', 'eax', 'lea', 'esp', 'esp'], tags=[3]),
 TaggedDocument(words=['lea', 'eax', 'edx', 'ecx', 'inc', 'ecx', 'mov', 'eax', 'esp', 'eax', 'var_', 'mov', 'eax', 'eax', 'mov', 'esp', 'ecx', 'var_', 'eax', 'cmp', 'ecx', 'esi'], tags=[4]),
 TaggedDocument(words=['mov', 'ecx', 'edi', 'lea', 'edx', 'esp

In [14]:
test_corpus

[['mov',
  'eax',
  'esp',
  'arg_',
  'sub',
  'esp',
  'ch',
  'push',
  'esi',
  'push',
  'edi',
  'mov',
  'edi',
  'ecx',
  'push',
  'ch',
  'mov',
  'edi',
  'eax',
  'lea',
  'eax',
  'esp',
  'var_',
  'push',
  'push',
  'eax',
  'mov',
  'dword',
  'ptr',
  'edi',
  'call',
  'mov',
  'esi',
  'esp',
  'arg_',
  'add',
  'esp',
  'ch',
  'cmp',
  'esi',
  'fh'],
 ['push',
  'fh',
  'push',
  'esi',
  'push',
  'offset',
  'atoomanylinkpro',
  'push',
  'offset',
  'adevcolorengine',
  'push',
  'push',
  'push',
  'call',
  'sub_',
  'dfb',
  'add',
  'esp',
  'ch'],
 ['xor', 'ecx', 'ecx', 'test', 'esi', 'esi'],
 ['mov',
  'edx',
  'esp',
  'arg_',
  'lea',
  'eax',
  'esp',
  'var_',
  'sub',
  'edx',
  'eax',
  'lea',
  'esp',
  'esp'],
 ['lea',
  'eax',
  'edx',
  'ecx',
  'inc',
  'ecx',
  'mov',
  'eax',
  'esp',
  'eax',
  'var_',
  'mov',
  'eax',
  'eax',
  'mov',
  'esp',
  'ecx',
  'var_',
  'eax',
  'cmp',
  'ecx',
  'esi'],
 ['mov',
  'ecx',
  'edi',
  'lea',
  

In [23]:
def train(ftrain):
    # 实例化Doc2Vec模型
    model = Doc2Vec(vector_size=100, window=3, cbow_mean=1, min_count=1)
    # 更新现有的word2vec模型
    model.build_vocab(ftrain)  # 使用数据建立单词表
    model.train(ftrain, total_examples=model.corpus_count, epochs=10)  # 训练模型，更新模型参数
    model.save(train_vec)
    return model

In [24]:
model_dm = train(train_corpus)

In [25]:
model_dm

In [26]:
# 模型训练完成后，可以用来生成一段文本的paragraph vector。
test_document = ['only', 'you', 'can', 'prevent', 'forest', 'fires']
test_vector = model_dm.infer_vector(test_document)
print(test_vector)

[-4.7894651e-03 -8.2983077e-04  2.8778493e-04  3.0024545e-03
  1.1931390e-03  4.3671937e-03  4.3317759e-03 -3.0365714e-03
 -4.1633840e-03  1.5181542e-03 -4.0454785e-03  2.1743667e-03
 -4.7096252e-04  4.6953082e-04 -3.6203149e-03  1.7196953e-04
 -3.9689187e-03  3.9184536e-03 -3.9677094e-03  4.2712311e-03
 -3.1901735e-03  4.3931184e-03 -6.8672985e-04 -4.9743743e-04
  4.2401687e-03 -1.4482358e-03  2.9019369e-03 -6.5437495e-04
 -2.4012476e-04  4.0361290e-03  1.3057578e-03  1.0324896e-03
  9.1733696e-04 -3.8609481e-03 -4.2408835e-03  1.8265301e-03
  2.3419566e-03  5.5308046e-04 -8.2617492e-04 -2.7447417e-03
  3.7275637e-03  4.3135923e-03  3.1245637e-03  1.8159061e-03
 -2.5134096e-03 -1.7492175e-05  1.5979176e-03 -8.6839555e-04
  3.5195954e-03 -6.1190454e-04  3.4785252e-03  4.9149869e-03
  3.6978102e-03 -4.0740413e-03  4.1750069e-03  4.4236733e-03
  6.6577614e-04 -1.5538249e-03  1.8911123e-04 -1.8605309e-03
  4.6625328e-03 -4.3749176e-03 -1.2566018e-03 -3.3547082e-03
 -4.8421617e-03 -6.68662

## PV-DBOW

In [ ]:
# # word2vec-skip在输入为paragraph token/id